In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [ ]:
import torch

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import (
    analyse_single_wsc, generate_full_sentences, find_missing_wsc_words_in_corpus_vocab, winograd_test
)

from src.wsc_parser import generate_df, generate_json

### Sentences with issues:

In [ ]:
still_in_english = [
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the spot under the tree was hot.",
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the dog was cooler.",
    "I couldn't put the pot on the shelf because the shelf was too tall.",
    "I couldn't put the pot on the shelf because the pot was too high.",
    "There is a pillar between me and the stage, and I can't see around the stage.",
    "There is a pillar between me and the stage, and I can't see the pillar.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear the subway.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear over the announcement.",
#     "The large ball crashed right through the table because the table was made of steel.",
#     "The large ball crashed right through the table because the large ball was made of styrofoam."
]

In [ ]:
subs_not_working = [
    'Jim fez um sinal para o barman e apontou para sua taça vazia.',
    'Jim fez um sinal para o barman e apontou para jim chave do banheiro.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Bill "Eu primeiro!" era lento.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Dan "Eu primeiro!" era mais rápido.',
    'Tom disse "Check" para Ralph enquanto ele movia Ralph bispo.',
    'Tom disse "Check" para Ralph enquanto ele tomava Tom bispo.',
    'John contratou Bill para cuidar Bill.',
    'John foi contratado por Bill para cuidar John.',
    'Ele os abaixou quando o vento parou.',
    'Ele os abriu quando o vento parou.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que matá- las.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que protegê- las.',
    'O caminho até o lago estava interditado, então nós não podíamos usá-o lago.',
    'O caminho até o lago estava interditado, então nós não podíamos alcançá-o caminho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o artigo lê quando chega do trabalho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o livro escreve quando chega do trabalho.',
    'Como estava chovendo, eu carreguei a revista na minha mochila para mantê-a mochila seca.',
    'Como estava chovendo, eu carreguei a revista sobre a minha mochila para mantê-a revista seca.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a peça tem desde criança.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a flauta ama desde criança.',
    'Joe vendeu sua casa e comprou uma nova a alguns quilômetros de distância. Ele se mudará dela na quinta-feira.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar removêo telhado.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar consertáo posto.',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia inseri-la .',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia removê-la.',
    'Há uma fenda na parede. É possível enxergar o jardim através a parede.',
    'Há uma fenda na parede. É possível enxergar o jardim atrás a fenda.',
    # ok mas estranho:
    'Eu usei um pano velho para limpar o alicate, e então o alicate coloquei no lixo.',
    'Eu usei um pano velho para limpar o alicate, e então o pano coloquei na gaveta.',
'    Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais jovens punimos.',
    'Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais velhos protegemos.'
    # "policia estavam", errado
    'Os bombeiros chegaram depois da polícia porque a polícia estavam vindo de muito longe.',
    'Os bombeiros chegaram antes da polícia porque os bombeiros estavam vindo de muito longe.',
    'O homem levantou o menino sobre sua cabeça.',
    'O homem levantou o menino até sua beliche.',
    'Alongando suas costas, a mulher sorriu para a menina.',
    'Dando um tapinha em suas costas, a mulher sorriu para a menina.',
    'Billy chorou porque Toby não queria aceitar seu brinquedo.',
    'Billy chorou porque Toby não queria compartilhar seu brinquedo.',
    'Lily conversou com Donna, quebrando sua promessa.',
    'Lily conversou com Donna, tirando sua concentração.',
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar empático.',
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar severo.',
    'Enquanto Ollie carregava Tommy subindo a longa escada em espiral, suas pernas doíam.',
    'Enquanto Ollie carregava Tommy subindo a longa escada em espiral, suas pernas balançavam.',
    'O pai carregava o menino dormindo em seu colo',
    'O pai carregava o menino dormindo em seu berço',
    'A mulher estava segurando a menina contra seu peito',
    'A mulher estava segurando a menina contra seu desejo.',
    'Este livro introduziu Shakespeare a Ovídio; ele foi uma grande influência em sua escrita.',
    'Este livro introduziu Shakespeare a Goethe; ele foi uma grande influência em sua escrita.',
    'Este livro introduziu Shakespeare a Ovídio; ele era uma excelente seleção de sua escrita.',
    'Este livro introduziu Shakespeare a Goethe; ele era uma excelente seleção de sua escrita.'
]

In [ ]:
df = generate_df(still_in_english)
df['correct_sentence'], df['incorrect_sentence'] = zip(*df.apply(generate_full_sentences, axis=1))

In [ ]:
df['correct_sentence'].replace({'alice': 'Alice'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'alice': 'Alice'}, inplace=True, regex=True)
df['correct_sentence'].replace({'fred': 'Fred'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'fred': 'Fred'}, inplace=True, regex=True)
df['correct_sentence'].replace({'mary': 'Mary'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'mary': 'Mary'}, inplace=True, regex=True)
df['correct_sentence'].replace({'anne': 'Anne'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'anne': 'Anne'}, inplace=True, regex=True)
df['correct_sentence'].replace({'jim': 'Jim'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'jim': 'Jim'}, inplace=True, regex=True)
df['correct_sentence'].replace({'O golpista': 'o golpista'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'O golpista': 'o golpista'}, inplace=True, regex=True)
df['correct_sentence'].replace({'O tio do Joe': 'o tio do Joe'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'O tio do Joe': 'o tio do Joe'}, inplace=True, regex=True)
df['correct_sentence'].replace({'desenho da tina': 'desenho da Tina'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'desenho da tina': 'desenho da Tina'}, inplace=True, regex=True)
df['correct_sentence'].replace({'desenho do sam': 'desenho do Sam'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'desenho do sam': 'desenho do Sam'}, inplace=True, regex=True)

In [ ]:
generate_json(df)